In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder 

C:\Users\Shend\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The dataset that contains ECG and HR data.

In [2]:
dataframe_hrv= pd.read_csv('C:/Users/Shend/Desktop/dataframe_hrv.csv')
dataframe_hrv = dataframe_hrv.reset_index(drop=True)

Changing stress values to 1, if the value is 0.5 or higher.
Changing stress values to 0, if the value is less than 0.5

In [3]:
def fix_stress_labels(df='',label_column='stress'):
    df['stress'] = np.where(df['stress']>=0.5, 1, 0)
    display(df["stress"].unique())
    return df
dataframe_hrv = fix_stress_labels(df=dataframe_hrv)
dataframe_hrv.head(5)

array([0, 1], dtype=int64)

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,...,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,...,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,101.978261,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,...,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,104.957447,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,...,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,...,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,88.829787,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,...,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


Extracting the values from the main dataframe and saving them to x and y.

x contains the values of ECG and HR.

y contains the values of Stress(0 or 1), meaning not stressed or stressed.

In [4]:
x = dataframe_hrv.iloc[:,[0,2]].values
y = dataframe_hrv.iloc[:,[10]].values
x[0]

array([-1.97368400e-03,  7.78157895e+01])

Now we HotEncode the data to work better with the algorithm. It transforms categorical features to a format that works better
with classification and regression algorithms.

In [5]:
oneHot = OneHotEncoder()
oneHot.fit(x)
x_final = oneHot.transform(x).toarray()
oneHot.fit(y)
y_final = oneHot.transform(y).toarray()
alpha, epochs = 0.0035, 500  #Learning rate, and the number of epochs
m, n = x_final.shape

C:\Users\Shend\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Shend\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

We create the model , by defining placeholders X and Y so we can feed our training examples x and y into the optimizer during
the training process.
We also create W and b which will be optimized through gradient descent optimizer.

In [6]:
X = tf.placeholder(tf.float32, [None, n], name="X")
Y = tf.placeholder(tf.float32, [None, 2], name="Y")

W = tf.Variable(tf.zeros([n, 2]), name="W")
b = tf.Variable(tf.zeros([2]), name="b")

Instructions for updating:
Colocations handled automatically by placer.


We delcare the Hypothesis, Cost function, Optimizer and Global Variables Initializer.

In [7]:
# Hypothesis 
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b)) 

# Sigmoid Function 
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits = Y_hat, labels = Y) 

# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = alpha).minimize(cost) 

# Global Variables Initializer 
init = tf.global_variables_initializer()

Do the training process inside a tensorflow session.
Also, save the model.

In [8]:
with tf.Session() as sess:
    sess.run(init)
    cost_history, accuracy_history = [], [] # Lists for storing cost and accuracy for every epoch
    for epoch in range(epochs):   # Iterating through epochs
        cost_per_epoch = 0
        sess.run(optimizer, feed_dict = {X: x_final, Y: y_final}) # Running the optimizer
        c = sess.run(cost, feed_dict = {X: x_final, Y: y_final})  # Cost on current epoch
        correct_prediction = tf.equal(tf.argmax(Y_hat, 1),        
                                     tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # Accuracy on current epoch
        cost_history.append(sum(sum(c))) # Store the cost 
        accuracy_history.append(accuracy.eval({X: x_final, Y : y_final}) * 100) # Store the accuracy
        
        #Displaying the results on current Epoch
        if epoch % 100 == 0 and epoch != 0:
            print("Epoch " + str(epoch) + " Cost: " + str(cost_history[-1]))
        
    Weight = sess.run(W) # Optimized Weight 
    Bias = sess.run(b)   # Optimized Bias 
      
    # Final Accuracy 
    correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
                                      tf.argmax(Y, 1)) 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,  
                                             tf.float32)) 
    print("\nAccuracy:", accuracy_history[-1], "%")
    tf.saved_model.simple_save(sess,'C:/Users/Shend/my_save',
                               inputs={"input" : X},
                               outputs={"predictor": Y_hat})

Epoch 100 Cost: 5696.05224609375
Epoch 200 Cost: 5675.92333984375
Epoch 300 Cost: 5662.44091796875
Epoch 400 Cost: 5653.22314453125

Accuracy: 57.544201612472534 %
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


AssertionError: Export directory already exists. Please specify a different export directory: C:/Users/Shend/my_save

Convert the saved model to tflite format, which is needed for using it on mobile applications.

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model(PATH)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:/Users/Shend/my_save\variables\variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input
INFO:tensorflow: tensor name: X:0, shape: (-1, 124), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: predictor
INFO:tensorflow: tensor name: Sigmoid_1:0, shape: (-1, 2), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from C:/Users/Shend/my_save\variables\variables
Instructions for updating:


1716